We'll download the dataset first:

In [1]:
import keras
from keras.datasets import cifar10

(X_train, Y_train), (X_validation, Y_validation) = cifar10.load_data()

Y_train = keras.utils.to_categorical(Y_train, 10)
Y_validation = keras.utils.to_categorical(Y_validation, 10)

Next, we'll instantiate a DataGenerator, which will normalize the train and test datasets, and will provide data augmentation during training:

In [2]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True)

# Apply z-normalization on the training set
data_generator.fit(X_train)

# Standardize the validation set
X_validation = data_generator.standardize(X_validation.astype('float32'))

Then, we'll define the CNN model:

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization

model = Sequential(layers=[
    Conv2D(32, (3, 3),
           padding='same',
           input_shape=X_train.shape[1:]),
    BatchNormalization(),
    Activation('gelu'),
    Conv2D(32, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('gelu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('gelu'),
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('gelu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3)),
    BatchNormalization(),
    Activation('gelu'),
    Conv2D(128, (3, 3)),
    BatchNormalization(),
    Activation('gelu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),

    Flatten(),
    Dense(10, activation='softmax')
])

We'll define the optimization parameters:

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0

Finally, we'll run the training and evaluation:

In [5]:
batch_size = 50

model.fit(
    x=data_generator.flow(x=X_train,
                          y=Y_train,
                          batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=100,
    verbose=1,
    validation_data=(X_validation, Y_validation),
    workers=4)

Epoch 1/100
1000/1000 [==============================] - 48s 44ms/step - loss: 1.8719 - accuracy: 0.3366 - val_loss: 1.8343 - val_accuracy: 0.3659
Epoch 2/100
1000/1000 [==============================] - 46s 46ms/step - loss: 1.5062 - accuracy: 0.4576 - val_loss: 1.4372 - val_accuracy: 0.4911
Epoch 3/100
1000/1000 [==============================] - 45s 45ms/step - loss: 1.3326 - accuracy: 0.5243 - val_loss: 1.3803 - val_accuracy: 0.5374
Epoch 4/100
1000/1000 [==============================] - 45s 45ms/step - loss: 1.2309 - accuracy: 0.5651 - val_loss: 1.2723 - val_accuracy: 0.5633
Epoch 5/100
1000/1000 [==============================] - 45s 45ms/step - loss: 1.1577 - accuracy: 0.5904 - val_loss: 1.2233 - val_accuracy: 0.5912
Epoch 6/100
1000/1000 [==============================] - 46s 46ms/step - loss: 1.1059 - accuracy: 0.6117 - val_loss: 1.1585 - val_accuracy: 0.6079
Epoch 7/100
1000/1000 [==============================] - 46s 46ms/step - loss: 1.0636 - accuracy: 0.6268 - val_loss: 1